In [1]:
import csv
import os
import subprocess
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
def run_lighthouse(url):
    cmd = f"lighthouse {url} --output=json --quiet"
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode != 0:
        print(f"Error running Lighthouse command for {url}: {result.stderr.decode()}")
        return None
    else:
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {url}: {e}")
            return None

In [3]:
def generate_report(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(url)
    report = run_lighthouse(url)
    
    driver.quit()
    
    return report

In [4]:
def save_report(report, filename, url, report_id):
    if report is not None:
        important_scores = {
            'id': report_id,
            'url': url,
            'performance': report['categories']['performance']['score'] * 100,
            'accessibility': report['categories']['accessibility']['score'] * 100,
            'best-practices': report['categories']['best-practices']['score'] * 100,
            'seo': report['categories']['seo']['score'] * 100
        }
        
        # Read the existing content if the file exists
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError:
                    data = {'lighthouseReports': []}
        else:
            data = {'lighthouseReports': []}
        
        # Append the new report to the list
        data['lighthouseReports'].append(important_scores)
        
        # Write the updated data back to the file
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
            print(f"Report saved for {filename}")
        
        # Print scores to the console
        print("Lighthouse Performance Metrics:")
        for category, score in important_scores.items():
            if category not in ['id', 'url']:
                print(f"{category.capitalize()} score: {score}")
    else:
        print(f"No report generated for {filename}")

In [5]:
if __name__ == "__main__": 
    with open('adrian_url.csv', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        
        next(csvreader)
        
        for report_id, row in enumerate(csvreader):
            url = 'https://' + row[0] + '/'
            print(url)
            report = generate_report(url)
            # filename = f"lighthouse_report_{url.replace('https://', '').replace('/', '_')}.json"
            save_report(report, "lighthouse_report.json", url, report_id)


https://pokemon.com/
Report saved for lighthouse_report.json
Lighthouse Performance Metrics:
Performance score: 51.0
Accessibility score: 72.0
Best-practices score: 56.99999999999999
Seo score: 67.0
https://viz.com/
Report saved for lighthouse_report.json
Lighthouse Performance Metrics:
Performance score: 46.0
Accessibility score: 83.0
Best-practices score: 56.99999999999999
Seo score: 85.0
